In [2]:
import plotly.graph_objects as go
import urllib, json

url = 'https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json'
response = urllib.request.urlopen(url)
data = json.loads(response.read())

# override gray link colors with 'source' colors
opacity = 0.4
# change 'magenta' to its 'rgba' value to add opacity
data['data'][0]['node']['color'] = ['rgba(255,0,255, 0.8)' if color == "magenta" else color for color in data['data'][0]['node']['color']]
data['data'][0]['link']['color'] = [data['data'][0]['node']['color'][src].replace("0.8", str(opacity))
                                    for src in data['data'][0]['link']['source']]

fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = "TWh",
    # Define nodes
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  data['data'][0]['node']['label'],
      color =  data['data'][0]['node']['color']
    ),
    # Add links
    link = dict(
      source =  data['data'][0]['link']['source'],
      target =  data['data'][0]['link']['target'],
      value =  data['data'][0]['link']['value'],
      label =  data['data'][0]['link']['label'],
      color =  data['data'][0]['link']['color']
))])

fig.update_layout(title_text="Energy forecast for 2050<br>Source: Department of Energy & Climate Change, Tom Counsell via <a href='https://bost.ocks.org/mike/sankey/'>Mike Bostock</a>",
                  font_size=10)
fig.show()

In [ ]:
import pickle

with open("Results/robust_partitions_f.npy, "rb") as f:
    data = pickle.load(f)

print(type(data))
print(data)


<class 'collections.defaultdict'>
defaultdict(<class 'list'>, {'scales': [0.01, 0.012328467394420659, 0.01519911082952934, 0.01873817422860384, 0.023101297000831605, 0.02848035868435802, 0.03511191734215131, 0.04328761281083057, 0.0533669923120631, 0.06579332246575682, 0.08111308307896872, 0.1, 0.12328467394420659, 0.1519911082952934, 0.1873817422860384, 0.23101297000831603, 0.2848035868435802, 0.3511191734215131, 0.43287612810830595, 0.533669923120631, 0.6579332246575682, 0.8111308307896873, 1.0, 1.232846739442066, 1.5199110829529332, 1.873817422860385, 2.31012970008316, 2.848035868435802, 3.511191734215131, 4.328761281083057, 5.336699231206313, 6.5793322465756825, 8.111308307896872, 10.0, 12.32846739442066, 15.199110829529348, 18.73817422860385, 23.101297000831604, 28.48035868435802, 35.11191734215131, 43.287612810830616, 53.366992312063125, 65.79332246575683, 81.11308307896873, 100.0, 123.28467394420659, 151.99110829529332, 187.3817422860383, 231.0129700083158, 284.8035868435805, 35

In [7]:
import numpy as np

In [21]:
dict=np.load('Pre_processed_Data/mapping_dict.npy', allow_pickle=True).item() 

In [22]:
dict

{0: '1A',
 1: '1A',
 2: '1A',
 3: '1A',
 4: '1A',
 5: '1A',
 6: '1A',
 7: '1A',
 8: '1A',
 9: '1A',
 10: '1A',
 11: '1A',
 12: '1A',
 13: '1A',
 14: '1A',
 15: '1A',
 16: '1A',
 17: '1A',
 18: '1A',
 19: '1A',
 20: '1A',
 21: '1A',
 22: '1A',
 23: '1B',
 24: '1B',
 25: '1B',
 26: '1B',
 27: '1B',
 28: '1B',
 29: '1B',
 30: '1B',
 31: '1B',
 32: '1B',
 33: '1B',
 34: '1B',
 35: '1B',
 36: '1B',
 37: '1B',
 38: '1B',
 39: '1B',
 40: '1B',
 41: '1B',
 42: '1B',
 43: '1B',
 44: '1B',
 45: '1B',
 46: '1B',
 47: '1B',
 48: '2A',
 49: '2A',
 50: '2A',
 51: '2A',
 52: '2A',
 53: '2A',
 54: '2A',
 55: '2A',
 56: '2A',
 57: '2A',
 58: '2A',
 59: '2A',
 60: '2A',
 61: '2A',
 62: '2A',
 63: '2A',
 64: '2A',
 65: '2A',
 66: '2A',
 67: '2A',
 68: '2A',
 69: '2A',
 70: '2A',
 71: '2B',
 72: '2B',
 73: '2B',
 74: '2B',
 75: '2B',
 76: '2B',
 77: '2B',
 78: '2B',
 79: '2B',
 80: '2B',
 81: '2B',
 82: '2B',
 83: '2B',
 84: '2B',
 85: '2B',
 86: '2B',
 87: '2B',
 88: '2B',
 89: '2B',
 90: '2B',
 91: '2B'

In [13]:
np.load('Results/robust_partitions_f.npy')[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 5, 6, 0, 4, 1, 4, 2, 7, 8, 3], dtype=int64)

In [10]:
np.load('Results/robust_partitions_f.npy')[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 1, 2, 0, 5, 1, 5, 1, 3, 4, 5], dtype=int64)

In [11]:
np.load('Results/robust_partitions_f.npy')[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 0, 2, 1, 2, 1, 4, 3, 2], dtype=int64)